### Piece Dynamics Tester

This notebook uses a saved model and test files from the MusicNet dataset to compute the predicted dynamics for these extra pieces.

In [ ]:
import re
import os

import numpy as np
from matplotlib import pyplot as plt
import essentia.standard as ess
import arff
import torch
import torch.nn as nn
import torch.nn.functional as F

import expression_modeling as m

labelsDir = 'data/musicnet/test_labels'
audioDir = 'data/musicnet/test_data'

def toOneHot(arr, index, nom_vals):
    if len(nom_vals) > 2:
        rows, cols = arr.shape
        newarr = np.zeros((rows, cols+len(nom_vals)-1), dtype=object)
        #print('oneHot: index=' + str(index) + '; vals=' + str(nom_vals))
        for _ in range(len(nom_vals)-1):
            d.attributes.insert(index,(d.attributes[index]))
        newarr[:,0:index] = arr[:,0:index]
        newarr[:,index+len(nom_vals):] = arr[:,index+1:]
        for i in range(0,len(nom_vals)):
            newarr[arr[:,index] == nom_vals[i],index+i] = 1
        return newarr
    else:
        aux = np.zeros((arr.shape[0],1))
        aux[arr[:,index] == nom_vals[0], 0] = 1
        arr[:,index] = aux[:,0]
        return arr

def resultForIndex(i):
        x = np.linspace(0, 1, len(d.moreVals[i][0]))
        ytest = (arr[i,-3]*moments[0,1]+moments[0,0])*x**2 + (arr[i,-2]*moments[1,1]+moments[1,0])*x + (arr[i,-1]*moments[2,1]+moments[2,0])
        y = (pred[i,0]*moments[0,1]+moments[0,0])*x**2 + (pred[i,1]*moments[1,1]+moments[1,0])*x + (pred[i,2]*moments[2,1]+moments[2,0])
        return (d.moreVals[i][0], ytest, y)
    
# filter only pieces which include the violin
dset = [csv for csv in os.listdir(labelsDir) if re.search(r'^(.*?,){2}\s*?41\s*?,(.*?,){3}', open(os.path.join(labelsDir, csv), 'r').read(), re.MULTILINE)]
dset = dset[0:1]
#tabla = []
outfile = None
for csv in dset:
    
    loader = ess.AudioLoader(filename=os.path.join(audioDir, csv.replace('.csv', '.wav')))
    audio, srate = loader()[0:2]

    loudnessAlg = ess.LoudnessEBUR128(hopSize=0.1, sampleRate=srate)
    levels = loudnessAlg(audio)[0]

    notearray = np.genfromtxt(os.path.join(labelsDir, csv), delimiter=',', names=True, dtype=['i', 'i', 'i', 'i', 'f', 'f', '|U40'])

    # piece key estimation (only major and minor for now)
    isMajor, key, llhoodM, llhoodm = m.estimateKey(notearray['note'])
    mode = m.Mode.major if isMajor else m.Mode.minor

    piece = m.Piece(key=key, mode=mode, name=csv)
    piece.parts = m.buildNoteParts(notearray, levels, srate)
    
    d = m.toMotifDataset(piece, 41)
    arr = d.valsArray
    
    rows, cols = arr.shape
    i = 0
    while i < cols:
        if isinstance(d.attributes[i][1], list):
            arr = toOneHot(arr, i, d.attributes[i][1])
            rows, cols = arr.shape
            if len(d.attributes[i][1]) > 2:
                i = i + len(d.attributes[i][1]) - 1
        i = i + 1
    
    arr = arr.astype(np.dtype('f8'))
    
    #  remove unused attributes
    rows, cols = arr.shape
    newarr = np.empty((rows,cols-2), dtype=np.dtype('f8'))
    newarr[:,:-3] = arr[:,:-5]
    newarr[:,-3:] = arr[:,-3:]
    d.attributes.pop(-4)
    d.attributes.pop(-4)
    #idcols = arr[:,-5:-3].astype(int)
    arr = newarr
    
    # standardize features
    rows, cols = arr.shape
    moments = np.zeros((3,2)) # output mean and std for reverting predictions
    moments[:,0] = np.mean(arr[:,-3:], axis=0)
    moments[:,1] = np.std(arr[:,-3:], ddof=1, axis=0)
    for i in range(cols):
        if d.attributes[i][1] == 'REAL':
            arr[:,i] = (arr[:,i] - np.mean(arr[:,i])) / np.std(arr[:,i], ddof=1)

    # load model
    _, attrs = arr.shape
    attrs = attrs - 3 # 3 outputs
    class Net(nn.Module):

        def __init__(self):
            super(Net, self).__init__()
            self.fc1 = nn.Linear(attrs, attrs//2 + 1)
            self.fc2 = nn.Linear(attrs//2 + 1, attrs//2 + 1)
            self.fc3 = nn.Linear(attrs//2 + 1, 3)

        def forward(self, x):
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.fc3(x)
            return x
    
    model = Net()
    sd = torch.load('results/mlp.pth')
    model.load_state_dict(sd)
    
    
    # run model
    print(arr.shape)
    pred = model(torch.Tensor(arr[:,:-3]))
    pred = pred.detach().numpy()
    print('r0: {pr}'.format(pr=np.corrcoef(pred[:,0], arr[:,-3])[0][1]))
    print('r1: {pr}'.format(pr=np.corrcoef(pred[:,1], arr[:,-2])[0][1]))
    print('r2: {pr}'.format(pr=np.corrcoef(pred[:,2], arr[:,-1])[0][1]))
    
    s = pred.shape[0]
    mse = np.zeros((s,1))
    for i in range(s):
         lvls, _, y = resultForIndex(i)
         mse[i] = np.sum((y - lvls)**2)

    print('Note-level RMS Error: ' + str(np.sqrt(np.mean(mse))))
    
    lvls, ytest, y = resultForIndex(0)
    x = np.linspace(0, 1, len(lvls))
    coefs = np.polyfit(x, lvls, 2)

    ycalc = coefs[0]*x**2 + coefs[1]*x + coefs[2]
    plt.plot(x, lvls)
    plt.plot(x, ytest)
    plt.plot(x, y)
    plt.show()

In [ ]:
lvls, ytest, y = resultForIndex(20)
x = np.linspace(0, 1, len(lvls))
coefs = np.polyfit(x, lvls, 2)

ycalc = coefs[0]*x**2 + coefs[1]*x + coefs[2]
plt.plot(x, lvls)
plt.plot(x, ytest)
plt.plot(x, y)
plt.show()